# Background

2019 Novel Coronavirus (2019-nCoV) is a virus (more specifically, a coronavirus) identified as the cause of an outbreak of respiratory illness first detected in Wuhan, China. Early on, many of the patients in the outbreak in Wuhan, China reportedly had some link to a large seafood and animal market, suggesting animal-to-person spread. However, a growing number of patients reportedly have not had exposure to animal markets, indicating person-to-person spread is occurring. At this time, it’s unclear how easily or sustainably this virus is spreading between people - CDC

## Library

In [372]:
# Standard libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline


## For web scraping
import requests 
from bs4 import BeautifulSoup 
import urllib.request as ur

# Plotly visualization open source
import plotly.express as px
from plotly.offline import init_notebook_mode, iplot
import plotly.figure_factory as ff # create_table for categorical variables
import cufflinks
cufflinks.go_offline()  
cufflinks.set_config_file(world_readable=True, theme='pearl')

import plotly.graph_objs as go # graphs on objects
import chart_studio.plotly as py # plotly library

# Research  

## This analysis aims to answer the following questions:
#### 1. Which countries flatten the curve faster than others? 

#### 2. What are they doing differently than other countries? Mask culture, # Hospitial, # of medical staff, #Ventilator, and day starting lockdown play a key?

#### 3. Is wearing mask without social distancing more effective than social distancing (Lockdown)?



## Proposed metrics 

Success= confirmed case/population , recovered/total case, death/total case? 

flatten the curve: day to be taken to get to the point of peak and have 3 consistent flatten rate of less than 5% variation up


# Data Collection
Population, mask culture, number of hospital, # medical staff, # Ventilator, # social distancing start date

### Covid19 Data

https://www.kaggle.com/sudalairajkumar/novel-corona-virus-2019-dataset

From World Health Organization - On 31 December 2019, WHO was alerted to several cases of pneumonia in Wuhan City, Hubei Province of China. The virus did not match any other known virus. This raised concern because when a virus is new, we do not know how it affects people.



In [352]:
df = pd.read_csv('covid_19_data.csv')

In [353]:
df.head()

,SNo,ObservationDate,Province/State,Country,Last Update,Confirmed,Deaths,Recovered
0,1,1/22/2020,Anhui,Mainland China,1/22/2020 17:00,1,0,0
1,2,1/22/2020,Beijing,Mainland China,1/22/2020 17:00,14,0,0
2,3,1/22/2020,Chongqing,Mainland China,1/22/2020 17:00,6,0,0
3,4,1/22/2020,Fujian,Mainland China,1/22/2020 17:00,1,0,0
4,5,1/22/2020,Gansu,Mainland China,1/22/2020 17:00,0,0,0


In [354]:
df['ObservationDate']=pd.to_datetime(df['ObservationDate'])


In [355]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10671 entries, 0 to 10670
Data columns (total 8 columns):
SNo                10671 non-null int64
ObservationDate    10671 non-null datetime64[ns]
Province/State     5715 non-null object
Country            10671 non-null object
Last Update        10671 non-null object
Confirmed          10671 non-null int64
Deaths             10671 non-null int64
Recovered          10671 non-null int64
dtypes: datetime64[ns](1), int64(4), object(3)
memory usage: 667.1+ KB


In [357]:
# Total confirmed case in the US as of 3/31/2020
df[(df['ObservationDate']=='03/31/2020') &(df['Country']=='US')].sum()['Confirmed']

188172

In [332]:
df

,SNo,ObservationDate,Province/State,Country/Region,Last Update,Confirmed,Deaths,Recovered,recover_rate
0,1,2020-01-22,Anhui,Mainland China,1/22/2020 17:00,1.0,0.0,0.0,0.000000
1,2,2020-01-22,Beijing,Mainland China,1/22/2020 17:00,14.0,0.0,0.0,0.000000
2,3,2020-01-22,Chongqing,Mainland China,1/22/2020 17:00,6.0,0.0,0.0,0.000000
3,4,2020-01-22,Fujian,Mainland China,1/22/2020 17:00,1.0,0.0,0.0,0.000000
4,5,2020-01-22,Gansu,Mainland China,1/22/2020 17:00,0.0,0.0,0.0,NaN
...,...,...,...,...,...,...,...,...,...
10666,10667,2020-03-31,Wyoming,US,2020-03-31 23:49:27,109.0,0.0,0.0,0.000000
10667,10668,2020-03-31,Xinjiang,Mainland China,2020-03-31 23:49:27,76.0,3.0,73.0,0.960526
10668,10669,2020-03-31,Yukon,Canada,2020-03-31 23:49:27,5.0,0.0,0.0,0.000000
10669,10670,2020-03-31,Yunnan,Mainland China,2020-03-31 23:49:27,182.0,2.0,172.0,0.945055


In [555]:
# group by country

df_country = df.groupby(['Country','ObservationDate']).sum()
df_country.head()

SNo  Confirmed  Deaths  Recovered
Country         ObservationDate                                    
 Azerbaijan     2020-02-28       2664          1       0          0
('St. Martin',) 2020-03-10       4675          2       0          0
Afghanistan     2020-02-24       2259          1       0          0
                2020-02-25       2352          1       0          0
                2020-02-26       2453          1       0          0

The period tracks from 2020-01-22 to 2020-03-31.

### Population data 

Source: https://www.worldometers.info/world-population/population-by-country/


In [370]:
# download and import population data
popu= pd.read_excel('population.xlsx')[['Country','Population']]

df_pop = pd.merge(df,popu,on='Country',how='inner')


### Medical supplies data - Web Scraping

Source : https://en.wikipedia.org/wiki/List_of_countries_by_hospital_beds

In [631]:
URL_track = "https://en.wikipedia.org/wiki/List_of_countries_by_hospital_beds"
p = requests.get(URL_track)
soup = BeautifulSoup(p2.content, 'html.parser')

In [632]:
country = soup2.find_all('td')
ls_country = []
for i in country:
    ls_country.append(list(i))

In [633]:
iter_list = list(zip(*[iter(ls_country)]*13))

In [634]:
med_df = pd.DataFrame(iter_list[:41])
med_df.columns =['Rank','Country','Continent','2013','2014','2015','2016','2017','Actual_change','Proportional_change','occupancy','ICCU-CCB beds','Ventilators']

In [635]:
med_df.head()

,Rank,Country,Continent,2013,2014,2015,2016,2017,Actual_change,Proportional_change,occupancy,ICCU-CCB beds,Ventilators
0,[1\n],"[[[<img alt="""" class=""thumbborder"" data-file-h...",[Asia\n],[13.3\n],[13.21\n],[13.17\n],[13.11\n],[13.05\n],[−0.06\n],[−0.48\n],[75.5\n],"[7.3, [[[17]]], \n]","[32,586 (2009), [[[18]]], \n]"
1,[2\n],"[[[<img alt="""" class=""thumbborder"" data-file-h...",[Asia\n],[10.92\n],[11.59\n],[11.61\n],[11.98\n],[12.27\n],[0.34\n],[2.85\n],[\n],"[10.6, [[[17]]], \n]","[9,795, [[[19]]], \n]"
2,[3\n],"[[[<img alt="""" class=""thumbborder"" data-file-h...",[Europe\n],[9.07\n],[8.81\n],[8.35\n],[8.16\n],[8.05\n],[−0.26\n],[−3.04\n],[\n],"[8.3, [[[20]]], \n]","[40,000, [[[21]]], \n]"
3,[4\n],"[[[<img alt="""" class=""thumbborder"" data-file-h...",[Europe\n],[8.28\n],[8.23\n],[8.13\n],[8.06\n],[8.00\n],[−0.07\n],[−0.86\n],[79.8\n],"[29.2, [[[22]]], \n]","[25,000, [[[23]]], \n]"
4,[5\n],"[[[<img alt="""" class=""thumbborder"" data-file-h...",[Europe\n],[7.64\n],[7.58\n],[7.54\n],[7.42\n],[7.37\n],[−0.07\n],[−0.90\n],[73.8\n],"[21.8, [[[22]]], \n]","[2,500, [[[24]]], \n]"


### Here I decided to clean it on Excel instead

In [731]:
# year refers to hospital bed per 1000 people
# ICU-CCB beds/100,000 inhabitants
med_df = pd.read_excel('medical_df.xlsx')  
med_df.head()

,Country,Continent,2013,2014,2015,2016,2017,"Change from previous year, average","ICU-CCB beds/100,000 inhabitants",Mask
0,Japan,Asia,13.30,13.21,13.17,13.11,13.05,−0.06,7.3,Y
1,South Korea,Asia,10.92,11.59,11.61,11.98,12.27,0.34,10.6,Y
2,Russia,Europe,9.07,8.81,8.35,8.16,8.05,−0.26,8.3,N
3,Germany,Europe,8.28,8.23,8.13,8.06,8.00,−0.07,29.2,N
4,Austria,Europe,7.64,7.58,7.54,7.42,7.37,−0.07,21.8,N


In [735]:
med_df[2017] =med_df[2017].astype(float)               #transform the data type to integer
med_df['Change from previous year, average']=med_df[2017]-med_df[2016]    # Here I used the available 2017 data to estimate the annual growth rate


In [736]:
med_df.dropna(subset=['ICU-CCB beds/100,000 inhabitants'],axis=0,inplace=True)         
med_df['ICU-CCB beds/100,000 inhabitants'] =med_df['ICU-CCB beds/100,000 inhabitants'].astype(float)

In [737]:
#hospital bed per 1000 people
med_df['2018'] = med_df[2017]+ med_df['Change from previous year, average']  # Reverse engineering to estimate 2018
med_df['2019'] = med_df['2018']+ med_df['Change from previous year, average'] # Reverse engineering to estimate 2019 

In [738]:
temp = str(med_df['Country']).replace('\xa0','').split('\n')         # First I transform the series into string and clean out noises
emp = [i[6:].strip() for i in temp]
med_df['Country']= emp[:-1]          


In [739]:
all_df = pd.merge(df_pop,med_df,on='Country',how='inner')              # Join the data from 2rd df with 3th df
all_df['Est_hos_bed']=all_df['Population']/1000*all_df['2019']              # Estimated total hospital bed
all_df['Est_icu_bed']=all_df['Population']/100000 *all_df['ICU-CCB beds/100,000 inhabitants'] # Estimated total ICU bed
all_df.head()

,SNo,ObservationDate,Province/State,Country,Last Update,Confirmed,Deaths,Recovered,Population,Continent,...,2015,2016,2017,"Change from previous year, average","ICU-CCB beds/100,000 inhabitants",Mask,2018,2019,Est_hos_bed,Est_icu_bed
0,1,2020-01-22,Anhui,Mainland China,1/22/2020 17:00,1,0,0,1439323776,Asia,...,3.82,4.05,4.34,0.29,3.6,Y,4.63,4.92,7.081473e+06,51815.655936
1,2,2020-01-22,Beijing,Mainland China,1/22/2020 17:00,14,0,0,1439323776,Asia,...,3.82,4.05,4.34,0.29,3.6,Y,4.63,4.92,7.081473e+06,51815.655936
2,3,2020-01-22,Chongqing,Mainland China,1/22/2020 17:00,6,0,0,1439323776,Asia,...,3.82,4.05,4.34,0.29,3.6,Y,4.63,4.92,7.081473e+06,51815.655936
3,4,2020-01-22,Fujian,Mainland China,1/22/2020 17:00,1,0,0,1439323776,Asia,...,3.82,4.05,4.34,0.29,3.6,Y,4.63,4.92,7.081473e+06,51815.655936
4,5,2020-01-22,Gansu,Mainland China,1/22/2020 17:00,0,0,0,1439323776,Asia,...,3.82,4.05,4.34,0.29,3.6,Y,4.63,4.92,7.081473e+06,51815.655936


# Data Exploration

In [740]:
# Filter out total confirmed cases as of 3/31/2020 by country

top_confirm=all_df[(all_df['ObservationDate']=='03/31/2020')][['Country','Confirmed']].groupby("Country").sum().sort_values(by='Confirmed',ascending=False)

top_confirm.head()

,Confirmed
Country,
US,188172
Italy,105792
Spain,95923
Mainland China,81524
Germany,71808


In [741]:
fig = go.Figure()
for i in top_confirm.index:
    fig.add_trace(go.Line(x=all_df[(all_df['Country']==i)].groupby('ObservationDate').sum().index, 
                          y=all_df[(all_df['Country']==i)].groupby('ObservationDate').sum()['Confirmed'],name=i))
fig.update_layout(
    title="<b>Total confirmed case  <b> ")

fig.show()

In [742]:
all_df['confirm_rate'] =all_df['Confirmed']/all_df['Population']
fig = go.Figure()
for i in top_confirm.index:
    fig.add_trace(go.Line(x=all_df[(all_df['Country']==i)].groupby('ObservationDate').sum().index, 
                          y=all_df[(all_df['Country']==i)].groupby('ObservationDate').sum()['confirm_rate'],name=i))
fig.update_layout(
    title="<b>Confirmed case/Population <b> ")

fig.show()

In [743]:
fig = go.Figure()
for i in top_confirm.index:
    fig.add_trace(go.Line(x=all_df[(all_df['Country']==i)].groupby('ObservationDate').sum().index, 
                          y=all_df[(all_df['Country']==i)].groupby('ObservationDate').sum()['Recovered'],name=i))
fig.update_layout(
    title="<b>Recover case over time <b> ")

fig.show()

In [763]:

fig = go.Figure()
all_df['recover_rate']=all_df['Recovered']/all_df['Confirmed']

for i in top_confirm.index:
    fig.add_trace(go.Line(x=all_df[(all_df['Country']==i)].groupby('ObservationDate').sum().index, 
                          y=all_df[(all_df['Country']==i)].groupby('ObservationDate').sum()['recover_rate'],name=i))
    
fig.update_layout(
    title="<b>Recover rate over time <b> ")
fig.show()

Now I want to find out which countries already flatten the curve. The way I do it is to find the rate of change and see which countries has 0% 

In [745]:
all_df.groupby('Continent').mean()

,SNo,Confirmed,Deaths,Recovered,Population,2013,2014,2015,2016,2017,"Change from previous year, average","ICU-CCB beds/100,000 inhabitants",2018,2019,Est_hos_bed,Est_icu_bed,confirm_rate,recover_rate
Continent,,,,,,,,,,,,,,,,,,
Asia,3458.642623,1837.561885,61.290984,1002.847541,1.319255e+09,3.889605,4.143717,4.274721,4.485434,4.750975,0.265541,4.048033,5.016516,5.282057,6.381131e+06,48298.954209,0.000004,0.522503
Europe,5628.566978,2923.799065,166.200156,309.983645,3.381644e+07,5.165989,5.130298,4.967936,4.895374,4.811628,-0.083746,11.839174,4.727882,4.644136,1.939783e+05,4759.378682,0.000154,0.085440
North America,5647.814180,446.683288,7.352941,9.170238,2.845714e+08,2.843938,2.787585,2.752976,2.723374,2.689162,-0.034211,31.093222,2.654951,2.620740,7.588203e+05,96980.362452,0.000002,inf
Oceania,4646.787879,91.280303,0.484848,6.426768,2.549988e+07,3.740000,3.790000,3.820000,3.840000,3.870000,0.030000,9.100000,3.900000,3.930000,1.002145e+05,2320.489444,0.000004,0.221803


In [793]:
all_df[(all_df['ObservationDate']=='03/31/2020')].groupby('Country').sum().head()

,SNo,Confirmed,Deaths,Recovered,Population,2013,2014,2015,2016,2017,"Change from previous year, average","ICU-CCB beds/100,000 inhabitants",2018,2019,Est_hos_bed,Est_icu_bed,confirm_rate,recover_rate
Country,,,,,,,,,,,,,,,,,,
Australia,85042,4559,18,358,203999072,29.92,30.32,30.56,30.72,30.96,0.24,72.8,31.20,31.44,8.017164e+05,18563.915552,0.000179,0.570480
Austria,10367,10180,128,1095,9006398,7.64,7.58,7.54,7.42,7.37,-0.05,21.8,7.32,7.27,6.547651e+04,1963.394764,0.001130,0.107564
Belgium,10374,12775,705,1696,11589623,5.93,5.85,5.83,5.76,5.76,0.00,15.9,5.76,5.76,6.675623e+04,1842.750057,0.001102,0.132759
Canada,159121,8527,101,1592,566132310,40.65,40.05,39.15,38.70,37.80,-0.90,202.5,36.90,36.00,1.358718e+06,76427.861850,0.000226,inf
Denmark,31529,3039,90,77,17376606,9.21,8.07,7.59,7.80,7.83,0.03,20.1,7.86,7.89,4.570047e+04,1164.232602,0.000525,0.638219


In [800]:
# Here I want to know the # hospital bed per capita
fig = go.Figure()
select_date = '03/31/2020'
by_country_mean=all_df[(all_df['ObservationDate']==select_date)].groupby('Country').mean().sort_values(by='2019',ascending=False)


# Mean hospital bed by country
fig.add_trace(go.Bar(x=by_country_mean.index, 
                          y=by_country_mean['2019'],name='Estimated # of Hospital bed'))
# Mean Recover Rate  by country
fig.add_trace(go.Bar(x=by_country_mean.index, 
                          y=by_country_mean['recover_rate'],name ='Recover per confirmed cases'))

fig.add_trace(go.Bar(x=by_country_mean.index, 
                          y=by_country_sum['confirm_rate'],name ='confirmed cases per population'))



fig.update_layout(
    title="<b>Statistics as of {} <b> ".format(select_date),barmode='overlay')

fig.show()

In [882]:
by_country_mean['Est_hos_bed']

Country
Japan             1.635341e+06
South Korea       6.588090e+05
Germany           6.602175e+05
Russia            1.142667e+06
Austria           6.547651e+04
Hungary           6.820208e+04
Poland            2.490307e+05
Lithuania         1.715042e+04
Slovakia          3.221189e+04
France            3.798918e+05
Belgium           6.675623e+04
Hong Kong         4.048370e+04
Latvia            9.940263e+03
Mainland China    7.081473e+06
Estonia           6.035734e+03
Slovenia          9.396800e+03
Switzerland       3.885925e+04
Luxembourg        2.729264e+03
Greece            4.408952e+04
Australia         1.002145e+05
Norway            1.864907e+04
Portugal          3.456684e+04
Italy             1.934778e+05
Netherlands       5.277541e+04
Spain             1.388617e+05
Ireland           1.451709e+04
Turkey            2.471135e+05
Iceland           9.964296e+02
US                8.870871e+05
Denmark           1.523349e+04
Canada            9.058117e+04
Sweden            1.999654e+04


In [931]:
fig = go.Figure()

select_date = '03/31/2020'
# Adjust groupby instance
by_country_sum=all_df[(all_df['ObservationDate']==select_date)].groupby('Country').sum().sort_values(by='Confirmed')
by_country_mean=all_df[(all_df['ObservationDate']==select_date)].groupby('Country').mean().sort_values(by='Confirmed')

# cases  by mask
fig.add_trace(go.Bar(x=by_country_sum.index, 
                          y=by_country_sum['Recovered'],name='Recover case'))

fig.add_trace(go.Bar(x=by_country_sum.index, 
                          y=by_country_sum['Confirmed'],name='Confirmed case'))

fig.add_trace(go.Bar(x=by_country_sum.index, 
                          y=by_country_sum['Deaths'],name='Deaths case'))

fig.add_trace(go.Scatter(x=by_country_mean.index, 
                          y=by_country_mean['Est_icu_bed'],name='ICU beds'))

button_layer_1_height = 1.15
fig.update_layout(
    title="<b>Country Statistics as of {} <b> ".format(select_date),barmode='overlay',
updatemenus=list([
    dict(active=-1,
         buttons=list([ 
             dict(label = "All",
                 method = "update",
                 args = [{"visible": [True, True, True,True]},
                         {"title": "All"}]),
            dict(label = "Recover case",
                 method = "update",
                 args = [{"visible": [True, False, False,True]},
                         {"title": "Recover case"}]),
            dict(label = "Confirmed case",
                 method = "update",
                 args = [{"visible": [False, True, False,True]},
                         {"title": "Confirmed case"}]),
             dict(label = "Deaths",
                 method = "update",
                 args = [{"visible": [False, False, True,True]},
                         {"title": "Deaths"}])
        ]),
         direction="down",
            pad={"r": 10, "t": 10},
            showactive=True,
            x=0.1,
            xanchor="left",
            y=button_layer_1_height,
            yanchor="top"
    )
]))

fig.update_layout(barmode='overlay',
    annotations=[
        dict(text="Cases", x=0.04, xref="paper", y=1.1, yref="paper",
                             align="left", showarrow=False)
    ])

fig.show()

In [854]:
fig = go.Figure()

select_date = '03/31/2020'
# Adjust groupby instance
by_country_sum=all_df[(all_df['ObservationDate']==select_date)].groupby('Continent').sum().sort_values(by='Recovered',ascending=False)


# cases  by mask
fig.add_trace(go.Bar(x=by_country_sum.index, 
                          y=by_country_sum['Recovered'],name='Recover case'))

fig.add_trace(go.Bar(x=by_country_sum.index, 
                          y=by_country_sum['Confirmed'],name='Confirmed case'))

fig.add_trace(go.Bar(x=by_country_sum.index, 
                          y=by_country_sum['Deaths'],name='Deaths case'))



button_layer_1_height = 1.15
fig.update_layout(
    title="<b>Continent Statistics as of {} <b> ".format(select_date),barmode='overlay',
updatemenus=list([
    dict(active=-1,
         buttons=list([ 
             dict(label = "All",
                 method = "update",
                 args = [{"visible": [True, True, True]},
                         {"title": "All"}]),
            dict(label = "Recover case",
                 method = "update",
                 args = [{"visible": [True, False, False]},
                         {"title": "Recover case"}]),
            dict(label = "Confirmed case",
                 method = "update",
                 args = [{"visible": [False, True, False]},
                         {"title": "Confirmed case"}]),
             dict(label = "Deaths",
                 method = "update",
                 args = [{"visible": [False, False, True]},
                         {"title": "Deaths"}])
        ]),
         direction="down",
            pad={"r": 10, "t": 10},
            showactive=True,
            x=0.1,
            xanchor="left",
            y=button_layer_1_height,
            yanchor="top"
    )
]))

fig.update_layout(
    annotations=[
        dict(text="Cases", x=0.04, xref="paper", y=1.1, yref="paper",
                             align="left", showarrow=False)
    ])

fig.show()

In [853]:
fig = go.Figure()

select_date = '03/31/2020'
# Adjust groupby instance
by_country_sum=all_df[(all_df['ObservationDate']==select_date)].groupby('Mask').sum().sort_values(by='Recovered',ascending=False)


# cases  by mask
fig.add_trace(go.Bar(x=by_country_sum.index, 
                          y=by_country_sum['Recovered'],name='Recover case'))

fig.add_trace(go.Bar(x=by_country_sum.index, 
                          y=by_country_sum['Confirmed'],name='Confirmed case'))

fig.add_trace(go.Bar(x=by_country_sum.index, 
                          y=by_country_sum['Deaths'],name='Deaths case'))



button_layer_1_height = 1.15
fig.update_layout(
    title="<b>Mask Culture Statistics as of {} <b> ".format(select_date),barmode='overlay',
updatemenus=list([
    dict(active=-1,
         buttons=list([ 
             dict(label = "All",
                 method = "update",
                 args = [{"visible": [True, True, True]},
                         {"title": "All"}]),
            dict(label = "Recover case",
                 method = "update",
                 args = [{"visible": [True, False, False]},
                         {"title": "Recover case"}]),
            dict(label = "Confirmed case",
                 method = "update",
                 args = [{"visible": [False, True, False]},
                         {"title": "Confirmed case"}]),
             dict(label = "Deaths",
                 method = "update",
                 args = [{"visible": [False, False, True]},
                         {"title": "Deaths"}])
        ]),
         direction="down",
            pad={"r": 10, "t": 10},
            showactive=True,
            x=0.1,
            xanchor="left",
            y=button_layer_1_height,
            yanchor="top"
    )
]))

fig.update_layout(
    annotations=[
        dict(text="Cases", x=0.04, xref="paper", y=1.1, yref="paper",
                             align="left", showarrow=False)
    ])

fig.show()

,SNo,ObservationDate,Province/State,Country,Last Update,Confirmed,Deaths,Recovered,Population,Continent,...,2017,"Change from previous year, average","ICU-CCB beds/100,000 inhabitants",Mask,2018,2019,Est_hos_bed,Est_icu_bed,confirm_rate,recover_rate
0,1,2020-01-22,Anhui,Mainland China,1/22/2020 17:00,1,0,0,1439323776,Asia,...,4.34,0.29,3.6,Y,4.63,4.92,7.081473e+06,51815.655936,6.947707e-10,0.000000
1,2,2020-01-22,Beijing,Mainland China,1/22/2020 17:00,14,0,0,1439323776,Asia,...,4.34,0.29,3.6,Y,4.63,4.92,7.081473e+06,51815.655936,9.726790e-09,0.000000
2,3,2020-01-22,Chongqing,Mainland China,1/22/2020 17:00,6,0,0,1439323776,Asia,...,4.34,0.29,3.6,Y,4.63,4.92,7.081473e+06,51815.655936,4.168624e-09,0.000000
3,4,2020-01-22,Fujian,Mainland China,1/22/2020 17:00,1,0,0,1439323776,Asia,...,4.34,0.29,3.6,Y,4.63,4.92,7.081473e+06,51815.655936,6.947707e-10,0.000000
4,5,2020-01-22,Gansu,Mainland China,1/22/2020 17:00,0,0,0,1439323776,Asia,...,4.34,0.29,3.6,Y,4.63,4.92,7.081473e+06,51815.655936,0.000000e+00,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6682,9278,2020-03-27,NaN,Turkey,3/27/2020 23:27,5698,92,42,84339067,Europe,...,2.81,0.06,47.1,N,2.87,2.93,2.471135e+05,39723.700557,6.756062e-05,0.007371
6683,9586,2020-03-28,NaN,Turkey,3/28/2020 23:11,7402,108,70,84339067,Europe,...,2.81,0.06,47.1,N,2.87,2.93,2.471135e+05,39723.700557,8.776478e-05,0.009457
6684,9897,2020-03-29,NaN,Turkey,3/29/2020 23:14,9217,131,105,84339067,Europe,...,2.81,0.06,47.1,N,2.87,2.93,2.471135e+05,39723.700557,1.092851e-04,0.011392
6685,10209,2020-03-30,NaN,Turkey,3/30/2020 22:58,10827,168,162,84339067,Europe,...,2.81,0.06,47.1,N,2.87,2.93,2.471135e+05,39723.700557,1.283747e-04,0.014963
